<a href="https://colab.research.google.com/github/horacefonseca/ped_ass_chatbot/blob/main/PedAss_MVP_Chatbot_ver5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# -*- coding: utf-8 -*-
"""PedAss_MVP_Chatbot_ver7_fixed.ipynb"""

# Installations
!pip install chatterbot chatterbot_corpus pytz

# Imports
import os
import re
import csv
import random
from datetime import datetime, timedelta
import pytz
from chatterbot import ChatBot
from chatterbot.trainers import ListTrainer, ChatterBotCorpusTrainer

# Timezone setup
FL_TIMEZONE = pytz.timezone('America/New_York')

class DynamicScheduler:
    def __init__(self):
        self.initialize_doctors()
        self.appointments = []

    def initialize_doctors(self):
        self.doctors = {
            'Aguilar': {
                'working_days': ['Monday', 'Wednesday', 'Friday'],
                'working_hours': {'start': '8:00 AM', 'end': '4:00 PM'},
                'appointment_duration': 30,
                'specialty': 'General Pediatrics'
            },
            'Irias': {
                'working_days': ['Tuesday', 'Wednesday', 'Thursday', 'Friday'],
                'working_hours': {'start': '10:00 AM', 'end': '4:00 PM'},
                'appointment_duration': 30,
                'specialty': 'Neonatology'
            },
            'Villalobos': {
                'working_days': ['Wednesday'],
                'working_hours': {'start': '8:00 AM', 'end': '4:00 PM'},
                'appointment_duration': 30,
                'specialty': 'Adolescent Medicine'
            },
            'Chacon': {
                'working_days': ['Tuesday', 'Thursday'],
                'working_hours': {'start': '9:00 AM', 'end': '5:00 PM'},
                'appointment_duration': 30,
                'specialty': 'Allergy Specialist'
            }
        }

    def generate_availability(self, doctor_lastname, weeks_ahead=4):
        doctor = self.doctors.get(doctor_lastname)
        if not doctor:
            return {}

        availability = {}
        today = datetime.now(FL_TIMEZONE)
        time_format = "%I:%M %p"
        start_time = datetime.strptime(doctor['working_hours']['start'], time_format)
        end_time = datetime.strptime(doctor['working_hours']['end'], time_format)

        for day in range(weeks_ahead * 7):
            current_date = today + timedelta(days=day)
            weekday = current_date.strftime('%A')

            if weekday in doctor['working_days']:
                date_str = current_date.strftime('%d/%m/%Y')
                availability[date_str] = {
                    'times': [],
                    'weekday': weekday,
                    'date_obj': current_date
                }

                current_slot = start_time
                while current_slot + timedelta(minutes=doctor['appointment_duration']) <= end_time:
                    availability[date_str]['times'].append(current_slot.strftime(time_format))
                    current_slot += timedelta(minutes=doctor['appointment_duration'])

        # Remove booked appointments
        for appt in self.appointments:
            if appt['doctor'] == doctor_lastname and appt['date'] in availability:
                if appt['time'] in availability[appt['date']]['times']:
                    availability[appt['date']]['times'].remove(appt['time'])

        return availability

    def has_appointment_with_doctor(self, patient_name, doctor_name):
        """Check if patient has any existing appointment with this doctor"""
        return any(
            appt for appt in self.appointments
            if (appt['patient'].lower() == patient_name.lower()
                and appt['doctor'] == doctor_name)
        )

class PediatricChatBot:
    def __init__(self):
        # Initialize all data structures first
        self.faqs = {
            "hours": "Our clinic is open Monday-Friday from 8AM to 5PM",
            "location": "We're located at 9655 NW 41st Street, Doral, FL",
            "services": "We offer: Well-child visits, Sick visits, Vaccinations, and Lab tests",
            "payment": "We accept most major insurance plans, credit cards, and cash payments",
            "parking": "We have free parking available in front of the clinic",
            "phone": "You can reach us at +1-305-436-1563",
            "contact": "For inquiries, call +1-305-436-1563 or visit us at 9655 NW 41st Street"
        }
        self.doctor_info = {
            "Aguilar": {
                "schedule": "Monday, Wednesday, Friday from 8AM-4PM",
                "specialty": "General Pediatrics"
            },
            "Chacon": {
                "schedule": "Tuesday, Thursday from 9AM-5PM",
                "specialty": "Allergy Specialist"
            },
            "Villalobos": {
                "schedule": "Wednesday from 8AM-4PM",
                "specialty": "Adolescent Medicine"
            },
            "Irias": {
                "schedule": "Tuesday-Thursday, Friday from 10AM-4PM",
                "specialty": "Neonatology"
            }
        }

        self.scheduler = DynamicScheduler()
        self.current_state = "MAIN_MENU"
        self.current_booking = {}
        self.last_interaction_time = datetime.now()
        self.setup_chatbot()

    def setup_chatbot(self):
        self.chatbot = ChatBot(
            'PediatricDoralBot',
            logic_adapters=[
                {
                    'import_path': 'chatterbot.logic.BestMatch',
                    'default_response': "I didn't understand. Please try again or say 'menu' to see options."
                }
            ],
            preprocessors=[
                'chatterbot.preprocessors.clean_whitespace'
            ]
        )

        # Disable learning after training
        self.chatbot.storage.drop()

        # Training data
        trainer = ListTrainer(self.chatbot)

        # Only train basic greetings - rest handled by our logic
        trainer.train([
            "hi", "Hello! Welcome to Pediatric Associates Doral! How can I help you today?",
            "hello", "Hello! How can I help you today?",
            "hey", "Hi there! What can I do for you?",
            "menu", "Returning to main menu...\n" + self.show_main_menu(),
            "back", "Going back..."
        ])

    def check_session_timeout(self):
        """Reset to main menu after 5 minutes of inactivity"""
        if (datetime.now() - self.last_interaction_time).seconds > 300:
            self.current_state = "MAIN_MENU"
            self.current_booking = {}
            return "Session timed out. Returning to main menu."
        return None

    def handle_response(self, user_input):
        self.last_interaction_time = datetime.now()

        # Check for quit command first
        if user_input.lower() == 'quit':
            return self.confirm_quit()

        # Check for session timeout
        timeout_msg = self.check_session_timeout()
        if timeout_msg:
            return timeout_msg

        user_input = user_input.lower().strip()

        # Navigation commands
        if user_input in ['menu', 'back', 'start over']:
            self.current_state = "MAIN_MENU"
            return self.show_main_menu()

        # State handling
        if self.current_state == "MAIN_MENU":
            return self.handle_main_menu(user_input)
        elif self.current_state == "FAQ_MENU":
            return self.handle_faq(user_input)
        elif self.current_state == "BOOKING_FLOW":
            return self.handle_booking_flow(user_input)

        return self.default_response()

    def confirm_quit(self):
        return ("Are you sure you want to quit?\n"
                "Type 'yes' to confirm or anything else to continue.")

    def default_response(self):
        return ("I didn't understand. Please try again or say 'menu' to start over.\n"
                "You can also say 'quit' to exit at any time.")

    def show_main_menu(self):
        return ("Main Menu:\n"
                "1. Book an appointment\n"
                "2. FAQs\n"
                "3. Clinic information\n"
                "Type your choice or say 'quit' to exit:")

    def handle_main_menu(self, user_input):
        user_input = user_input.lower()

        if re.search(r'\b(book|schedule|make|appointment|reserve|1)\b', user_input):
            self.current_state = "BOOKING_FLOW"
            return self.start_booking()
        elif re.search(r'\b(faqs?|questions?|help|info|information|2)\b', user_input):
            self.current_state = "FAQ_MENU"
            return self.show_faq_menu()
        elif re.search(r'\b(clinic\s*info|about\s*clinic|clinic\s*details|3)\b', user_input):
            return self.show_clinic_info() + self.navigation_prompt()
        elif re.search(r'\b(doctor|physician|schedule|availability)\b', user_input):
            return self.show_doctor_schedules() + self.navigation_prompt()
        else:
            for key in self.faqs:
                if re.search(rf'\b{key}\b', user_input):
                    return self.faqs[key] + self.navigation_prompt()
            return self.default_response()

    def navigation_prompt(self):
        return "\n\n(You can say 'menu' to return or 'quit' to exit)"

    def show_doctor_schedules(self):
        info = "Doctor Schedules and Specialties:\n"
        for doctor, details in self.doctor_info.items():
            info += (f"- Dr. {doctor} ({details['specialty']}): "
                    f"Available {details['schedule']}\n")
        return info

    def show_faq_menu(self):
        return ("FAQs about:\n"
                "- Hours\n"
                "- Location\n"
                "- Services\n"
                "- Payment\n"
                "- Parking\n"
                "- Contact\n"
                "Ask your question or say 'back':")

    def handle_faq(self, user_input):
        user_input = user_input.lower()
        # Map variations to standard keys
        faq_mapping = {
            'hour': 'hours',
            'open': 'hours',
            'time': 'hours',
            'where': 'location',
            'address': 'location',
            'service': 'services',
            'offer': 'services',
            'pay': 'payment',
            'insurance': 'payment',
            'park': 'parking',
            'call': 'phone',
            'phone': 'phone',
            'contact': 'contact',
            'number': 'phone'
        }

        # Check mapped variations first
        for variation, key in faq_mapping.items():
            if variation in user_input:
                return self.faqs[key] + self.navigation_prompt()

        # Check direct matches
        for key in self.faqs:
            if key in user_input:
                return self.faqs[key] + self.navigation_prompt()

        if user_input == 'back':
            self.current_state = "MAIN_MENU"
            return self.show_main_menu()

        return ("I didn't understand. Please ask about hours, location, services, "
               "payment, parking, or contact info." + self.navigation_prompt())

    def show_clinic_info(self):
        return ("Clinic Information:\n"
                f"Name: Pediatric Associates Doral\n"
                f"Address: 9655 NW 41st Street, Doral, FL\n"
                f"Phone: +1-305-436-1563\n"
                f"Hours: Monday-Friday 8AM-5PM\n"
                f"Services: General pediatrics, Neonatology, Adolescent Medicine, Allergy Specialist")

    def start_booking(self):
        self.current_booking = {}
        doctor_list = "\n".join(
            f"- Dr. {doctor} ({details['specialty']})"
            for doctor, details in self.doctor_info.items()
        )
        return (f"Available doctors:\n{doctor_list}\n"
                "Please select a doctor by last name or say 'any'.\n"
                "You can say 'back' to return or 'quit' to exit:")

    def handle_booking_flow(self, user_input):
        if user_input.lower() == 'quit':
            return self.confirm_quit()
        if user_input.lower() == 'back':
            self.current_state = "MAIN_MENU"
            return self.show_main_menu()

        if not self.current_booking.get('doctor'):
            return self.process_doctor_selection(user_input)
        elif not self.current_booking.get('date'):
            return self.process_date_selection(user_input)
        elif not self.current_booking.get('time'):
            return self.process_time_selection(user_input)
        elif not self.current_booking.get('patient'):
            return self.process_patient_info(user_input)
        elif not self.current_booking.get('confirmed'):
            return self.process_confirmation(user_input)
        else:
            return self.finalize_booking()

    def process_doctor_selection(self, user_input):
        if user_input.lower() == 'back':
            self.current_state = "MAIN_MENU"
            return self.show_main_menu()

        if user_input.lower() == 'any':
            self.current_booking['doctor'] = "Aguilar"
            availability = self.scheduler.generate_availability("Aguilar")
        else:
            # Match any part of the doctor's name
            doctor = next((d for d in self.scheduler.doctors.keys()
                         if d.lower() in user_input.lower()), None)
            if not doctor:
                return ("Doctor not found. Available doctors:\n"
                        f"{self.show_doctor_schedules()}\n"
                        "Please try again or say 'back':")
            self.current_booking['doctor'] = doctor
            availability = self.scheduler.generate_availability(doctor)

        if not availability:
            return ("No availability found for this doctor. "
                    "Please try another doctor or say 'back':")

        next_available = list(availability.keys())[:5]
        formatted_dates = [
            f"{date} ({availability[date]['weekday']})"
            for date in next_available
        ]
        return (f"Dr. {self.current_booking['doctor']} is available on:\n"
                f"{', '.join(formatted_dates)}\n"
                "Please enter date (DD/MM/YYYY) or say 'back':")

    def process_date_selection(self, user_input):
        if user_input.lower() == 'back':
            self.current_booking.pop('doctor', None)
            return self.start_booking()

        try:
            input_date = datetime.strptime(user_input, '%d/%m/%Y')
            if input_date.date() < datetime.now().date():
                return "Date must be in future. Please try again:"
        except ValueError:
            return "Invalid date format (DD/MM/YYYY). Please try again:"

        doctor = self.current_booking['doctor']
        availability = self.scheduler.generate_availability(doctor)

        if user_input not in availability:
            return "Date not available. Please choose another or say 'back':"

        self.current_booking['date'] = user_input
        self.current_booking['weekday'] = availability[user_input]['weekday']
        available_times = availability[user_input]['times'][:10]
        return (f"Available times on {self.current_booking['weekday']} {user_input}:\n"
                f"{', '.join(available_times)}\n"
                "Please choose a time (e.g., '08:00 AM') or say 'back':")

    def process_time_selection(self, user_input):
        if user_input.lower() == 'back':
            self.current_booking.pop('date', None)
            return self.process_doctor_selection(self.current_booking['doctor'])

        doctor = self.current_booking['doctor']
        date = self.current_booking['date']
        availability = self.scheduler.generate_availability(doctor)

        try:
            normalized_time = self.normalize_time_input(user_input)
            if normalized_time not in availability.get(date, {}).get('times', []):
                return "Time not available. Please choose another or say 'back':"
            self.current_booking['time'] = normalized_time
            return "Please enter patient's full name or say 'back':"
        except ValueError:
            return "Invalid time format. Please use format like '08:00 AM' or say 'back':"

    def process_patient_info(self, user_input):
        if user_input.lower() == 'back':
            self.current_booking.pop('time', None)
            return self.process_date_selection(self.current_booking['date'])

        self.current_booking['patient'] = user_input.title()

        # Check for existing appointments with same doctor
        if self.scheduler.has_appointment_with_doctor(
            self.current_booking['patient'],
            self.current_booking['doctor']
        ):
            self.current_booking['duplicate_doctor'] = True
            return (f"❌ {self.current_booking['patient']} already has an appointment "
                    f"with Dr. {self.current_booking['doctor']}.\n"
                    "Would you like to book with a different doctor? (yes/no)")

        return self.show_booking_confirmation()

    def show_booking_confirmation(self):
        return (f"Confirm appointment for {self.current_booking['patient']}:\n"
                f"📅 {self.current_booking['weekday']}, {self.current_booking['date']} "
                f"at {self.current_booking['time']}\n"
                f"👨‍⚕️ Dr. {self.current_booking['doctor']}\n"
                "Type 'yes' to confirm or 'no' to cancel:")

    def process_confirmation(self, user_input):
        if user_input.lower() == 'back':
            self.current_booking.pop('patient', None)
            if self.current_booking.get('duplicate_doctor'):
                self.current_booking.pop('duplicate_doctor', None)
                self.current_booking.pop('doctor', None)
                return self.start_booking()
            return "Please enter patient's full name or say 'back':"

        if user_input.lower() in ['yes', 'y', 'confirm']:
            if self.current_booking.get('duplicate_doctor'):
                # Handle the case where they said "yes" to booking with different doctor
                self.current_booking.pop('duplicate_doctor', None)
                self.current_booking.pop('doctor', None)
                return self.start_booking()
            self.current_booking['confirmed'] = True
            return self.finalize_booking()
        elif user_input.lower() in ['no', 'n', 'cancel']:
            self.current_state = "MAIN_MENU"
            self.current_booking = {}
            return "Appointment cancelled. What would you like to do next?"
        else:
            return "Please answer 'yes' to confirm or 'no' to cancel:"

    def finalize_booking(self):
        # Save appointment
        self.scheduler.appointments.append({
            'doctor': self.current_booking['doctor'],
            'date': self.current_booking['date'],
            'time': self.current_booking['time'],
            'patient': self.current_booking['patient'],
            'booked_at': datetime.now(FL_TIMEZONE).strftime('%Y-%m-%d %H:%M:%S')
        })

        confirmation_msg = random.choice([
            f"✅ Appointment confirmed for {self.current_booking['patient']}!",
            f"✅ All set, {self.current_booking['patient']}! Your visit is scheduled.",
            f"✅ Got it! We'll see you on {self.current_booking['date']}, {self.current_booking['patient']}!"
        ])

        confirmation = (
            f"{confirmation_msg}\n"
            f"👨‍⚕️ Doctor: Dr. {self.current_booking['doctor']}\n"
            f"📅 Date: {self.current_booking['weekday']}, {self.current_booking['date']}\n"
            f"⏰ Time: {self.current_booking['time']}\n"
            "ℹ️ Please arrive 15 minutes early.\n"
            "You can say 'menu' for more options or 'quit' to exit."
        )

        self.current_state = "MAIN_MENU"
        self.current_booking = {}
        return confirmation

    def normalize_time_input(self, time_str):
        """Convert various time formats to standard format"""
        time_str = time_str.upper().replace('.', '').strip()
        if ':' not in time_str:
            if 'AM' in time_str or 'PM' in time_str:
                time_part = time_str.replace('AM', '').replace('PM', '').strip()
                if len(time_part.split()) > 1:
                    time_part = time_part.split()[0]
                time_str = f"{time_part}:00 {time_str[-2:]}"
            else:
                time_str = f"{time_str}:00 AM"

        time_obj = datetime.strptime(time_str, "%I:%M %p")
        return time_obj.strftime("%I:%M %p")

def main():
    bot = PediatricChatBot()
    print("=== Pediatric Associates Doral ===")
    print("Now with enhanced conversation handling!")
    print("Type 'quit' anytime to exit or 'menu' to restart\n")
    print(bot.show_main_menu())

    while True:
        try:
            user_input = input("You: ").strip()

            # Handle quit confirmation
            if user_input.lower() == 'quit':
                print("Bot: Thank you! Have a great day.")
                break

            response = bot.handle_response(user_input)
            print(f"Bot: {response}")

        except KeyboardInterrupt:
            print("\nBot: Thank you! Have a great day.")
            break
        except Exception as e:
            print(f"Bot: Sorry, something went wrong. Let's start fresh.")
            print(bot.show_main_menu())

if __name__ == "__main__":
    main()

List Trainer: 10it [00:00, 556.66it/s]


=== Pediatric Associates Doral ===
Now with enhanced conversation handling!
Type 'quit' anytime to exit or 'menu' to restart

Main Menu:
1. Book an appointment
2. FAQs
3. Clinic information
Type your choice or say 'quit' to exit:
You: appo
Bot: I didn't understand. Please try again or say 'menu' to start over.
You can also say 'quit' to exit at any time.
You: appointment
Bot: Available doctors:
- Dr. Aguilar (General Pediatrics)
- Dr. Chacon (Allergy Specialist)
- Dr. Villalobos (Adolescent Medicine)
- Dr. Irias (Neonatology)
Please select a doctor by last name or say 'any'.
You can say 'back' to return or 'quit' to exit:
You: any
Bot: Dr. Aguilar is available on:
25/07/2025 (Friday), 28/07/2025 (Monday), 30/07/2025 (Wednesday), 01/08/2025 (Friday), 04/08/2025 (Monday)
Please enter date (DD/MM/YYYY) or say 'back':
You: 25/07/2025
Bot: Available times on Friday 25/07/2025:
08:00 AM, 08:30 AM, 09:00 AM, 09:30 AM, 10:00 AM, 10:30 AM, 11:00 AM, 11:30 AM, 12:00 PM, 12:30 PM
Please choose a 